# Analysis of Additional Risk Metrics for Bitcoin

## Setup

In [ ]:
from matplotlib import font_manager
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
# Set charts theme
sns.set_theme(style="darkgrid", rc={"grid.alpha": 0.33})
plt.style.use("dark_background")

# Save chart as png function
def save_chart_as_png(filename: str) -> None:
    plt.savefig(
        f"../images/{filename}.png",
        format="png",
        dpi=300,
        orientation="landscape",
        bbox_inches="tight",
    )

In [ ]:
# Get all dfs
def get_df(csv_basename: str) -> pd.DataFrame:
    # Get df from CSV with date as index
    return pd.read_csv(f"../data/{csv_basename}.csv", index_col="date", parse_dates=True)

df_btc = get_df("BTC")
df_us10y = get_df("US10Y")

## Bitcoin Yearly Risk-Adjusted Returns Over Time ⚖️

### Sharpe Ratio

In [ ]:
# Get YoY returns
# Get yearly bitcoin price df with first and last prices
df_btc_yearly = df_btc.groupby(df_btc.index.year)["price"].agg(
    first_price="first",
    last_price="last",
)
# Get YoY return
df_btc_yearly["price_change"] = (df_btc_yearly["last_price"] - df_btc_yearly["first_price"]) / df_btc_yearly["first_price"]

In [ ]:
# Get the volatility of monthly returns along with the number of months on yearly df
monthly_returns = df_btc["price"].resample("ME").ffill().pct_change() # Resample to monthly (month end)
monthly_stats = monthly_returns.groupby(monthly_returns.index.year).agg(
    volatility_m=("std"),
    num_months=("count"),
)
df_btc_yearly[["volatility_m", "num_months"]] = monthly_stats.reindex(df_btc_yearly.index)

In [ ]:
# Get risk free rate for each year from the 10-year US treasury yield average
df_btc_yearly["us10_yield_avg"] = df_us10y["yield"].groupby(df_us10y.index.year).mean()

In [ ]:
# Calculate Sharpe ratio, except for incomplete years (2010 and 2024)
df_btc_yearly.loc[df_btc_yearly["num_months"] == 12, "sharpe_ratio"] = round((df_btc_yearly["price_change"] - df_btc_yearly["us10_yield_avg"]) / df_btc_yearly["volatility_m"], 3)

In [ ]:
plt.figure(figsize=(10, 6))

# Create a custom palette for positive and negative ratios
palette_greens = LinearSegmentedColormap.from_list("positive",["#d4c334", "#40d434"])
palette_reds = LinearSegmentedColormap.from_list("negative", ["#d43438", "#d47034"])

# Get one barplot for the positive and another for the negative ratios while making sure they are in the correct order
ax = sns.barplot(data=df_btc_yearly[df_btc_yearly["sharpe_ratio"] >= 0], x="date", y="sharpe_ratio", order=df_btc_yearly[df_btc_yearly["sharpe_ratio"].notnull()].index, palette=palette_greens, hue="sharpe_ratio", legend=False)
ax = sns.barplot(data=df_btc_yearly[df_btc_yearly["sharpe_ratio"] < 0], x="date", y="sharpe_ratio", order=df_btc_yearly[df_btc_yearly["sharpe_ratio"].notnull()].index, palette=palette_reds, hue="sharpe_ratio", legend=False)

plt.yticks([-20, -10, 0, 10, 20, 30])

# Label each bar with its value
font_properties = font_manager.FontProperties(family="sans-serif", weight="bold", size=8)
for container in ax.containers:
    ax.bar_label(container, fmt="%.2f", padding=2.5, fontproperties=font_properties)

plt.title("Bitcoin Yearly Sharpe Ratio Over Time")
plt.xlabel(None)
plt.ylabel(None)

save_chart_as_png("3.2_BTC_yearly_sharpe")

In [ ]:
# Highest yearly Sharpe ratio
df_btc_yearly.loc[[df_btc_yearly["sharpe_ratio"].idxmax()]]

In [ ]:
# Lowest yearly Sharpe ratio
df_btc_yearly.loc[[df_btc_yearly["sharpe_ratio"].idxmin()]]

In [ ]:
# Create table with yearly Sharpe ratios stats
pd.DataFrame({
    "Average yearly Sharpe ratio": [round(df_btc_yearly["sharpe_ratio"].mean(), 4)],
    "Median yearly Sharpe ratio": [round(df_btc_yearly["sharpe_ratio"].median(), 4)],
    "Standard deviation": [round(df_btc_yearly["sharpe_ratio"].std(), 4)],
    "Min yearly Sharpe ratio": [round(df_btc_yearly["sharpe_ratio"].min(), 4)],
    "Max yearly Sharpe ratio": [round(df_btc_yearly["sharpe_ratio"].max(), 4)],
})

**Key takeaways:**
- ...

### Sortino Ratio

In [ ]:
# Get the volatility of monthly negative returns on yearly df
downside_monthly_returns = monthly_returns[monthly_returns < 0]
downside_monthly_volatility = downside_monthly_returns.groupby(downside_monthly_returns.index.year).std()
df_btc_yearly["downside_volatility_m"] = downside_monthly_volatility.reindex(df_btc_yearly.index)

In [ ]:
# Calculate Sortino ratio, except for incomplete years (2010 and 2024)
df_btc_yearly.loc[df_btc_yearly["num_months"] == 12, "sortino_ratio"] = round((df_btc_yearly["price_change"] - df_btc_yearly["us10_yield_avg"]) / df_btc_yearly["downside_volatility_m"], 3)

In [ ]:
plt.figure(figsize=(10, 6))

# Create a custom palette for positive and negative ratios
palette_greens = LinearSegmentedColormap.from_list("positive",["#d4c334", "#40d434"])
palette_reds = LinearSegmentedColormap.from_list("negative", ["#d43438", "#d47034"])

# Get one barplot for the positive and another for the negative ratios while making sure they are in the correct order
ax = sns.barplot(data=df_btc_yearly[df_btc_yearly["sortino_ratio"] >= 0], x="date", y="sortino_ratio", order=df_btc_yearly[df_btc_yearly["sortino_ratio"].notnull()].index, palette=palette_greens, hue="sortino_ratio", legend=False)
ax = sns.barplot(data=df_btc_yearly[df_btc_yearly["sortino_ratio"] < 0], x="date", y="sortino_ratio", order=df_btc_yearly[df_btc_yearly["sortino_ratio"].notnull()].index, palette=palette_reds, hue="sortino_ratio", legend=False)

plt.yticks([-50, 0, 50, 200, 300])

# Label each bar with its value
font_properties = font_manager.FontProperties(family="sans-serif", weight="bold", size=8)
for container in ax.containers:
    ax.bar_label(container, fmt="%.2f", padding=2.5, fontproperties=font_properties)

plt.title("Bitcoin Yearly Sortino Ratio Over Time")
plt.xlabel(None)
plt.ylabel(None)

save_chart_as_png("3.2_BTC_yearly_sortino")

In [ ]:
# Highest yearly Sortino ratio
df_btc_yearly.loc[[df_btc_yearly["sortino_ratio"].idxmax()]]

In [ ]:
# Lowest yearly Sortino ratio
df_btc_yearly.loc[[df_btc_yearly["sortino_ratio"].idxmin()]]

In [ ]:
# Create table with yearly Sortino ratios stats
pd.DataFrame({
    "Average yearly Sortino ratio": [round(df_btc_yearly["sortino_ratio"].mean(), 4)],
    "Median yearly Sortino ratio": [round(df_btc_yearly["sortino_ratio"].median(), 4)],
    "Standard deviation": [round(df_btc_yearly["sortino_ratio"].std(), 4)],
    "Min yearly Sortino ratio": [round(df_btc_yearly["sortino_ratio"].min(), 4)],
    "Max yearly Sortino ratio": [round(df_btc_yearly["sortino_ratio"].max(), 4)],
})

**Key takeaways:**
- ...

## Value at Risk (VaR) and Expected Shortfall (CVaR) 🚨

In [ ]:
# Get log price change
df_btc["price_change_log"] = np.log(df_btc["price"] / df_btc["price"].shift(1))

In [ ]:
# Calculate VaR and CVaR based for specific aggregated returns and confidence interval
def calculate_var_and_cvar(aggregated_returns: pd.Series, confidence_interval: float) -> tuple[float, float]:
    # Convert confidence interval to the corresponding percentile for VaR calculation
    percentile = (1 - confidence_interval) * 100
    
    # Calculate the historical VaR as the negative value at the specified percentile of aggregated returns
    var = -np.percentile(aggregated_returns, percentile).round(3)
    
    # Calculate CVaR as the negative mean of returns that are less than or equal to the calculated VaR
    cvar = -aggregated_returns[aggregated_returns <= -var].mean().round(3)
    
    return var, cvar

In [ ]:
# Get three VaR histograms using the list of aggregated returns for each timeframe
def show_three_var_histograms(aggregated_returns_list: list[pd.Series], method: str) -> None:
    colors = ["#40e0d0", "#4067e0", "#6b40e0"]
    titles = [f"1-quater {method} Returns", f"1-year {method} Returns", f"5-year {method} Returns"]
    
    fig, axes = plt.subplots(1, 3, figsize=(14, 5), sharey=True)
    
    for i, aggregated_returns in enumerate(aggregated_returns_list):
        sns.histplot(aggregated_returns, stat="probability", binwidth=0.1, binrange=(-1, 1), color=colors[i], edgecolor="white", alpha=0.75, ax=axes[i])
        
        axes[i].axvline(np.percentile(aggregated_returns, 5), color="orange", linewidth=1.5, linestyle="--", label="VaR at 95% Confidence Level")
        axes[i].axvline(np.percentile(aggregated_returns, 1), color="red", linewidth=1.5, linestyle="--", label="VaR at 99% Confidence Level")
    
        axes[i].set_xlim(-1, 1)
        axes[i].tick_params(axis="both", labelsize=10) 
        
        axes[i].set_title(f"Distribution of the {titles[i]}")
        axes[i].set_xlabel(None)
        
    axes[0].set_ylabel("Probability")
    
    axes[1].legend(loc="upper right", fontsize=10)
    
    plt.tight_layout()

### Historical Method

In [ ]:
# Get table with VaR and CVaR for a specific confidence interval for different time horizons using the historical method
time_horizons = [90, 365, 1825]
aggregated_returns_list = []
var_cvar_results = []

for time_horizon in time_horizons:
    # Calculate the rolling sum of log price changes for the specified time horizon
    aggregated_returns = df_btc["price_change_log"].rolling(window=time_horizon).sum().dropna()

    # Append aggregated returns to use in histograms
    aggregated_returns_list.append(aggregated_returns)
    
    # Calculate VaR and CVaR using the aggregated returns for 95% and 99% confidence interval
    var_95, cvar_95 = calculate_var_and_cvar(aggregated_returns, 0.95)
    var_99, cvar_99 = calculate_var_and_cvar(aggregated_returns, 0.99)
    
    # Append results to the list
    var_cvar_results.append({
        "Time Horizon": f"{time_horizon} days",
        "VaR (95%)": var_95,
        "CVaR (95%)": cvar_95,
        "VaR (99%)": var_99,
        "CVaR (99%)": cvar_99,
    })

pd.DataFrame(var_cvar_results)

In [ ]:
show_three_var_histograms(aggregated_returns_list, "Historical")

### Monte Carlo Method

In [ ]:
# Get table with VaR and CVaR for a specific confidence interval for different time horizons using the Monte Carlo method
mean = df_btc["price_change_log"].mean()
std = df_btc["price_change_log"].std()
num_simulations = 10_000

time_horizons = [90, 365, 1825]
aggregated_returns_list = []
var_cvar_results = []

for time_horizon in time_horizons:
    # Simulate future returns using a normal distribution (output is array of x days by y simulations)
    simulated_returns = np.random.normal(mean, std, (num_simulations, time_horizon))
    
    # Aggregate returns over the time horizon (sum x days of each simulation)
    aggregated_returns = simulated_returns.sum(axis=1)

    # Append aggregated returns to use in histograms
    aggregated_returns_list.append(aggregated_returns)
    
    # Calculate VaR and CVaR using the aggregated returns for 95% and 99% confidence interval
    var_95, cvar_95 = calculate_var_and_cvar(aggregated_returns, 0.95)
    var_99, cvar_99 = calculate_var_and_cvar(aggregated_returns, 0.99)
    
    # Append results to the list
    var_cvar_results.append({
        "Time Horizon": f"{time_horizon} days",
        "VaR (95%)": var_95,
        "CVaR (95%)": cvar_95,
        "VaR (99%)": var_99,
        "CVaR (99%)": cvar_99,
    })

pd.DataFrame(var_cvar_results)

In [ ]:
show_three_var_histograms(aggregated_returns_list, "Simulated")

### Monte Carlo Method (Using Values Since 2019)

In [ ]:
# Get table with VaR and CVaR for a specific confidence interval for different time horizons using the Monte Carlo method
mean = df_btc[df_btc.index.year >= 2019]["price_change_log"].mean()
std = df_btc[df_btc.index.year >= 2019]["price_change_log"].std()
num_simulations = 25_000

time_horizons = [90, 365, 1825]
aggregated_returns_list = []
var_cvar_results = []

for time_horizon in time_horizons:
    # Simulate future returns using a normal distribution (output is array of x days by y simulations)
    simulated_returns = np.random.normal(mean, std, (num_simulations, time_horizon))
    
    # Aggregate returns over the time horizon (sum x days of each simulation)
    aggregated_returns = simulated_returns.sum(axis=1)

    # Append aggregated returns to use in histograms
    aggregated_returns_list.append(aggregated_returns)
    
    # Calculate VaR and CVaR using the aggregated returns for 95% and 99% confidence interval
    var_95, cvar_95 = calculate_var_and_cvar(aggregated_returns, 0.95)
    var_99, cvar_99 = calculate_var_and_cvar(aggregated_returns, 0.99)
    
    # Append results to the list
    var_cvar_results.append({
        "Time Horizon": f"{time_horizon} days",
        "VaR (95%)": var_95,
        "CVaR (95%)": cvar_95,
        "VaR (99%)": var_99,
        "CVaR (99%)": cvar_99,
    })

pd.DataFrame(var_cvar_results)

In [ ]:
show_three_var_histograms(aggregated_returns_list, "Simulated")

save_chart_as_png("3.2_BTC_var")

**Key takeaways:**
- ...